# INDEX
## as per baseline
1. merge train and test
2. do some shit to understand data
3. *missing* >> engineer features on df
## other approach
4. group and median in train2
5. model building >> **doesn't run, problem with input shapes**
## build 'main' model
6. split df in train and test (now nonesense as no features are present)
7. build model
8. train and test

# TODO:
* imports and loading data
* remove NaN
* check out data
* look for correlations
    * scatterplots between all vars and y
* make dummy variables for categorical data
    * store type
    * [...]
* highlight relationship between sales and holidays
* make model
    * test different:
        * number of nodes
        * activation functions

## Importing - setup - load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# Set seed for reproducability 
np.random.seed(42)

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

concatenate data into pandas dataframe in order to pre-process data and create dummy vars.

In [ ]:
df = pd.concat([train,test],axis=0)
df.head(20)

In [ ]:
df.describe()

## fix missing values
now we fill in the missing values with 0, as they are all those referring to markdowns.
the process applies to 
* markdown columns
* weekly sale of test data, which can be safely set to zero as we will predict those value later on

In [ ]:
df.fillna(0, inplace=True)
df.isnull().sum()

In [ ]:
df.dtypes

next step is to prep the data (converting in to dummies) like in the example from [here](https://www.kaggle.com/erkkinool/bletchley-week-2-walmart-sales-forecast) and try to run a first model. after the baseline I'll try to get better performance.

In [ ]:
df.head(10)

# looking for correlations
to build some features i look for some correlation btween vars and sales. scatterplots are an easy solution.

In [ ]:
def scatterplots(feature, label):
    x = feature
    y = df['Weekly_Sales']
    plt.scatter(x, y)
    plt.ylabel('sales')
    plt.xlabel(label)
    plt.show()
    
fig = plt.gcf()
fig.set_size_inches(8, 8)

In [ ]:
headers = list(df)
labels = headers
scatterplots(df['CPI'], 'CPI')
scatterplots(df['Date'], 'Date') # date isn't readable in scatterplot. make it timeseries
scatterplots(df['Dept'], 'Dept')
scatterplots(df['Fuel_Price'], 'Fuel_Price')
scatterplots(df['IsHoliday'], 'IsHoliday')
scatterplots(df['MarkDown1'], 'MarkDown1')
scatterplots(df['MarkDown2'], 'MarkDown2')
scatterplots(df['MarkDown3'], 'MarkDown3')
scatterplots(df['MarkDown4'], 'MarkDown4')
scatterplots(df['MarkDown5'], 'MarkDown5')
scatterplots(df['Size'], 'Size')
scatterplots(df['Store'], 'Store')
scatterplots(df['Temperature'], 'Temperature')
scatterplots(df['Type'], 'Type')
scatterplots(df['Unemployment'], 'Unemployment')

### comment: 
the scatterplot approach worked halfway. some cool relationships there but would be worth to investigate further. Some observations:
* high correlation between holiday and sales. some outliers. don't think there is additional feature to be made of that.
* two big spikes around ?christmas? in sales. >> **make feature of that**, maybe for whole month. no need for every week
* relation between CPI. first block has more variability.... don't understand the sense of it
* high correlation between some dpt and sales. >> **make conditional hystogram like in example to investigate further**
* at some levels of fuel we have more variability. mmm...
* negative correlation btween markdown and sales?
* temperature can be split in three blocks. >> **make feature of it** medium temp is favorable for sales
* type B sells nmore than others. no feature there i think
* unemployment doesn't look interesting. maybe rising or dropping unemployment? there is probably some delay there though.



### TODO further research:
* sum of sales by year. could be a trend there. correlate sales trend with macro variables. macro variables are unlikely to be strongly correlated with weekly stuff, or maybe it's implicitly in the model?
* check out:
    * https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/discussion/8033
    * https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/discussion/8023
    * good simple model https://www.kaggle.com/c/see-click-predict-fix/discussion/6466#35490

# Simple model approach as per [here](https://www.kaggle.com/c/see-click-predict-fix/discussion/6466#35490)

creating month column. 
I have done it both on df and on new dataframe 'train2' which is just modified "train"

In [ ]:
# extract month from date
df['month'] = pd.DatetimeIndex(df['Date']).month

In [ ]:
df.head()

In [ ]:
# create train2. take median and groupby. get mean_vals
train2 = train
train2['month'] = pd.DatetimeIndex(train2['Date']).month
mean_vals = train2.groupby(['Store', 'Dept', 'month', 'IsHoliday']).median()
# replace NaN
mean_vals.fillna(0, inplace=True)

In [ ]:
mean_vals.dtypes

now let's try to run a **model** on it

In [ ]:
y1 = train2['Weekly_Sales'].values
X1 = train2.drop('Weekly_Sales',axis=1).values

In [ ]:
model = Sequential()

# std 3 layer network
model.add(Dense(units=320, input_dim=16, activation='tanh'))
model.add(Dense(units=160, activation='tanh'))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam',
              loss='mae',
              metrics=['acc'])

model.fit(X1, y1, epochs=10, batch_size= 2048)

# build a model
